## Coordinate Correction
This notebook is to convert the wrong labels of the rectified image into normal image coordinates. This file is run in colab, the data is stored in google drive, and only the algorithm is demonstrated here.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

All corrected labels will be saved in a file called `plan.csv`. The following is the coordinate correction of the `Narrabeen` dataset. If you need to correct the coordinates of other datasets, please modify the following code to the corresponding JPW parameter value.
```python
# Affine transformation parameters from JPW file
A = 6.1232340e-17
D = -1.0000000e+00
B = -1.0000000e+00
E = -6.1232340e-17
C = 3.4278421e+05
F = 6.2666314e+06
```

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiLineString, LineString
import re
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import os

# Affine transformation parameters from JPW file
A = 6.1232340e-17
D = -1.0000000e+00
B = -1.0000000e+00
E = -6.1232340e-17
C = 3.4278421e+05
F = 6.2666314e+06

# Read the CSV file
csv_file = 'segment_narraV2_c1.csv'
df = pd.read_csv(csv_file)

# Initialize plan.csv
output_columns = ['path', 'rectified', 'site', 'camera', 'type', 'obstruction', 'downward', 'low', 'shadow', 'label']
output_df = pd.DataFrame(columns=output_columns)

# Image path prefix
image_prefix = 'Argus narrabeen/'

# Record the number of times images are displayed
plot_count = 0

# Process all data
for index, row in df.iterrows():
    oblique_image = row['oblique_image']
    rectified_geometry = row['rectified_geometry']
    if not isinstance(oblique_image, str):
        continue

    # Replace timex with plan
    plan_image = oblique_image.replace('timex', 'plan')
    image_path = image_prefix + plan_image

    # Check if the image file exists
    if not os.path.exists(image_path):
        continue

    # Extract coordinates
    multiline_str = rectified_geometry
    multiline = re.findall(r'[-+]?\d*\.\d+|[-+]?\d+', multiline_str)
    if len(multiline) % 2 != 0:
        continue
    multiline = [(float(multiline[i]), float(multiline[i+1])) for i in range(0, len(multiline), 2)]

    # Apply affine transformation to convert coordinates to image coordinate system
    transformed_coords = [(A*x + B*y + C + 1000, - D*x + E*y + F + 100) for x, y in multiline]
    true_coords = [(A*x + B*y + 1000,  D*x - E*y + 100) for x, y in multiline]

    lines = [LineString(transformed_coords)]
    multi_line = MultiLineString(lines)
    true_line = MultiLineString([LineString(true_coords)])

    # Add to the output DataFrame
    new_row = {
        'path': image_path,
        'rectified': 1,
        'site': 'narrabn',
        'camera': 'c5',
        'type': 'timex',
        'obstruction': 0,
        'downward': 0,
        'low': 0,
        'shadow': 0,
        'label': true_line
    }
    output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)

    # Display the first ten images
    if plot_count < 10:
        # Read the image
        image = Image.open(image_path)
        gdf = gpd.GeoDataFrame(geometry=[multi_line], crs='EPSG:28356')
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.imshow(image, extent=[C, C + image.size[0], F, F + image.size[1]], origin='upper')
        gdf.plot(ax=ax, color='red', linewidth=2)
        ax.set_title('Coastline Overlay on Corrected Image')
        plt.show()
        plot_count += 1

# Save to a new CSV file
output_csv_file = 'plan.csv'
output_df.to_csv(output_csv_file, index=False)


Output hidden; open in https://colab.research.google.com to view.